In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import os
import json

# Define the path to the JSON file`
file_path = '/content/drive/MyDrive/train.json'
# Open the JSON file
with open(file_path, "r") as f:
    train_data = json.load(f)  # Load JSON contents into a variable

# Print the first entry to check structure
print(type(train_data))
print(list(train_data.keys()))

<class 'dict'>
['133', '73', '21', '145', '40', '22', '13', '67', '18', '78', '57', '122', '193', '101', '29', '79', '184', '3', '139', '191', '147', '10', '2', '131', '177', '92', '154', '164', '137', '134', '15', '116', '142', '31', '153', '158', '53', '160', '161', '187', '33', '119', '35', '174', '155', '4', '170', '24', '176', '107', '75', '27', '180', '143', '121', '166', '81', '138', '86', '14', '189', '60', '1', '43', '129', '163', '16', '49', '195', '71', '150', '62', '123', '126', '61', '103', '162', '168', '36', '118', '179', '185', '30', '156', '54', '91', '140', '66', '112', '19', '6', '84', '188', '59', '68', '152', '25', '186', '64', '167', '39', '148', '165', '135', '98', '169', '12', '190', '82', '93']


{<br/>
  "133": {<br/>
    "question": "how much oxazepam could cause an overdose?",<br/> // main question <br/>
    "multi_abs_summ": "Oxazepam is used to treat anxiety and symptoms of alcohol withdrawal...",<br/> // abstractive summary generated using multiple documents<br/>
    "multi_ext_summ": "Oxazepam is a medicine used to treat anxiety and symptoms of alcohol withdrawal...",<br/> // extractive summary generated using multiple documents<br/>
    "answers": {<br/>
      "133_Answer2": {<br/>
        "answer_abs_summ": "Oxazepam is used to treat anxiety and symptoms of alcohol withdrawal...",// abstractive summary generated using single doc, article and url //given below<br/>
        "answer_ext_summ": "Oxazepam is a medicine used to treat anxiety and symptoms of alcohol withdrawal...",// same but it's extractive summ<br/>
        "section": "What is Oxazepam overdose?: Oxazepam is a medicine...",<br/>
        "article": "Oxazepam overdose Benzodiazepine overdose Serax overdose...",<br/>
        "url": "https://www.nlm.nih.gov/medlineplus/ency/article/002516.htm",<br/>
        "rating": "3-Incomplete" // rating of the answer provided, this can be used to teach the model<br/>
      },<br/>
      "133_Answer3": {<br/>
        "answer_abs_summ": "...",<br/>
        "answer_ext_summ": "...",<br/>
        "section": "...",<br/>
        "article": "...",<br/>
        "url": "...",<br/>
        "rating": "..."<br/>
      }<br/>
    }<br/>
  }<br/>
}<br/>

In [3]:
queries = []
multi_abs_summaries = []
multi_ext_summaries = []
answer_abs_summaries  = []
answer_ext_summaries = []
detailed_sections = []
articles = []
source_urls = []
answer_ratings = []
answer_ids = []

for question_id, question_data in train_data.items():
  questions_text = question_data["question"]
  multi_abs_summ= question_data.get("multi_abs_summ","N/A")
  multi_ext_summ = question_data.get("multi_ext_summ","N/A")

  if "answers" in question_data:
    for answer_id, answer_data in question_data["answers"].items():
      queries.append(questions_text)
      multi_abs_summaries.append(multi_abs_summ)
      multi_ext_summaries.append(multi_ext_summ)
      answer_ids.append(answer_id)

      # now we start putting the single doc summaries

      answer_abs_summ = answer_data.get("answer_abs_summ","N/A")
      answer_ext_summ = answer_data.get("answer_ext_summ", "N/A")
      detailed_section = answer_data.get("section","N/A")
      url  = answer_data.get("url", "N/A")
      rating = answer_data.get("rating", "N/A")
      answer_abs_summaries.append(answer_abs_summ)
      answer_ext_summaries.append(answer_ext_summ)
      detailed_sections.append(detailed_section)
      articles.append(answer_data.get("article","N/A"))
      answer_ratings.append(rating)
      source_urls.append(url)

In [4]:
import pandas as pd
#converting the created lists into a df
df = pd.DataFrame({
    "Query": queries,
    "Answer ID": answer_ids,  # NEW COLUMN
    "Multi-Doc Abstractive Summary": multi_abs_summaries,
    "Multi-Doc Extractive Summary": multi_ext_summaries,
    "Answer Abstractive Summary": answer_abs_summaries,
    "Answer Extractive Summary": answer_ext_summaries,
    "Detailed Section": detailed_sections,
    "Full Article": articles,
    "Source URL": source_urls,
    "Answer Rating": answer_ratings
})

csv_path = "/content/cleaned_train_data_with_ids.csv"
df.to_csv(csv_path, index=False)

df.head(10)

,Query,Answer ID,Multi-Doc Abstractive Summary,Multi-Doc Extractive Summary,Answer Abstractive Summary,Answer Extractive Summary,Detailed Section,Full Article,Source URL,Answer Rating
0,how much oxazepam could cause an overdose?,133_Answer2,Oxazepam is used to treat anxiety and symptoms...,Oxazepam is a medicine used to treat anxiety a...,Oxazepam is used to treat anxiety and symptoms...,Oxazepam is a medicine used to treat anxiety a...,What is Oxazepam overdose?: Oxazepam is a medi...,Oxazepam overdose Benzodiazepine overdose Ser...,https://www.nlm.nih.gov/medlineplus/ency/artic...,3-Incomplete
1,how much oxazepam could cause an overdose?,133_Answer3,Oxazepam is used to treat anxiety and symptoms...,Oxazepam is a medicine used to treat anxiety a...,Oxazepam is a controlled substance that can re...,"In case of overdose, call your local poison co...",Oxazepam: Oxazepam may increase the risk of se...,Oxazepam IMPORTANT WARNING: Oxazepam may incr...,https://medlineplus.gov/druginfo/meds/a682050....,3-Incomplete
2,how much oxazepam could cause an overdose?,133_Answer9,Oxazepam is used to treat anxiety and symptoms...,Oxazepam is a medicine used to treat anxiety a...,"In case of overdose, call your local poison co...","In case of overdose, call your local poison co...",Oxazepam (In case of emergency/overdose): In c...,Oxazepam IMPORTANT WARNING: Oxazepam may incr...,https://medlineplus.gov/druginfo/meds/a682050....,3-Incomplete
3,Project. Hi my name is and I'm doing a school...,73_Answer1,Leukemia treatments are based on the patients...,Your doctor determines your leukemia treatment...,Leukemia treatments are based on the patients...,Your doctor determines your leukemia treatment...,Leukemia (Treatment): Treatment for your leuke...,Leukemia Overview Leukemia is cancer of the b...,https://www.mayoclinic.org/diseases-conditions...,3-Incomplete
4,My cousin she is leber amaurosis she need help...,21_Answer1,Leber congenital amaurosis is associated with ...,Leber congenital amaurosis is an eye disorder ...,Leber congenital amaurosis is associated with ...,Leber congenital amaurosis is an eye disorder ...,Leber congenital amaurosis: Leber congenital a...,"Leber congenital amaurosis amaurosis, Leber c...",https://ghr.nlm.nih.gov/condition/leber-congen...,3-Incomplete
5,My cousin she is leber amaurosis she need help...,21_Answer8,Leber congenital amaurosis is associated with ...,Leber congenital amaurosis is an eye disorder ...,Treatments for Leber congenital amaurosis are ...,The resources below provide information about ...,Leber congenital amaurosis (Treatment): The re...,Leber congenital amaurosis LCA Congenital abs...,https://rarediseases.info.nih.gov/diseases/634...,3-Incomplete
6,how to prevent diabetes,145_Answer10,Keeping an ideal body weight and an active lif...,Diabetes (Is there anything I can do to preven...,"At this time, researchers do not know how to p...",Diabetes (Is there anything I can do to preven...,Diabetes (Is there anything I can do to preven...,Diabetes Overview Diabetes is a disease in wh...,https://www.womenshealth.gov/a-z-topics/diabetes,3-Incomplete
7,how to prevent diabetes,145_Answer11,Keeping an ideal body weight and an active lif...,Diabetes (Is there anything I can do to preven...,Work with your health care team to adjust your...,Preventing Diabetes Problems (How can I preven...,Preventing Diabetes Problems (How can I preven...,Preventing Diabetes Problems Heart Disease an...,https://www.niddk.nih.gov/health-information/d...,3-Incomplete
8,how to prevent diabetes,145_Answer2,Keeping an ideal body weight and an active lif...,Diabetes (Is there anything I can do to preven...,Type 1 diabetes cannot be prevented. There is ...,How to prevent Type 1 diabetes?: Type 1 diabet...,How to prevent Type 1 diabetes?: Type 1 diabet...,Type 1 diabetes Insulin-dependent diabetes Ju...,https://www.nlm.nih.gov/medlineplus/ency/artic...,3-Incomplete
9,how to prevent diabetes,145_Answer3,Keeping an ideal body weight and an 

In [5]:
from google.colab import files
#files.download("/content/cleaned_train_data_with_ids.csv")

In [6]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#Loading validation dataset

In [17]:
import os
import json

# Define the path to the JSON file`
file_path = '/content/drive/MyDrive/validation.json'
# Open the JSON file
with open(file_path, "r") as f:
    validation_data = json.load(f)  # Load JSON contents into a variable

# Print the first entry to check structure
print(type(validation_data))
print(list(validation_data.keys()))

<class 'dict'>
['85', '115', '106', '132', '5', '55', '23', '32', '105', '95', '108', '159', '17', '46', '113', '157']


In [18]:
queries = []
multi_abs_summaries = []
multi_ext_summaries = []
answer_abs_summaries  = []
answer_ext_summaries = []
detailed_sections = []
articles = []
source_urls = []
answer_ratings = []
answer_ids = []

for question_id, question_data in validation_data.items():
  questions_text = question_data["question"]
  multi_abs_summ= question_data.get("multi_abs_summ","N/A")
  multi_ext_summ = question_data.get("multi_ext_summ","N/A")

  if "answers" in question_data:
    for answer_id, answer_data in question_data["answers"].items():
      queries.append(questions_text)
      multi_abs_summaries.append(multi_abs_summ)
      multi_ext_summaries.append(multi_ext_summ)
      answer_ids.append(answer_id)

      # now we start putting the single doc summaries

      answer_abs_summ = answer_data.get("answer_abs_summ","N/A")
      answer_ext_summ = answer_data.get("answer_ext_summ", "N/A")
      detailed_section = answer_data.get("section","N/A")
      url  = answer_data.get("url", "N/A")
      rating = answer_data.get("rating", "N/A")
      answer_abs_summaries.append(answer_abs_summ)
      answer_ext_summaries.append(answer_ext_summ)
      detailed_sections.append(detailed_section)
      articles.append(answer_data.get("article","N/A"))
      answer_ratings.append(rating)
      source_urls.append(url)

In [19]:
import pandas as pd
#converting the created lists into a df
df = pd.DataFrame({
    "Query": queries,
    "Answer ID": answer_ids,  # NEW COLUMN
    "Multi-Doc Abstractive Summary": multi_abs_summaries,
    "Multi-Doc Extractive Summary": multi_ext_summaries,
    "Answer Abstractive Summary": answer_abs_summaries,
    "Answer Extractive Summary": answer_ext_summaries,
    "Detailed Section": detailed_sections,
    "Full Article": articles,
    "Source URL": source_urls,
    "Answer Rating": answer_ratings
})

csv_path = "/content/cleaned_validation_data_with_ids.csv"
df.to_csv(csv_path, index=False)

df.head(10)

,Query,Answer ID,Multi-Doc Abstractive Summary,Multi-Doc Extractive Summary,Answer Abstractive Summary,Answer Extractive Summary,Detailed Section,Full Article,Source URL,Answer Rating
0,subjective vertigo Can macular degeneration in...,85_Answer1,"Dizziness has many possible causes, including ...","Dizziness has many possible causes, including ...","Dizziness has many possible causes, including ...","Dizziness has many possible causes, including ...",Dizziness: Dizziness is a term used to describ...,Dizziness Overview Dizziness is a term used t...,https://www.mayoclinic.org/diseases-conditions...,3-Incomplete
1,subjective vertigo Can macular degeneration in...,85_Answer2,"Dizziness has many possible causes, including ...","Dizziness has many possible causes, including ...",Macular degeneration is a disease that destroy...,"Macular degeneration, or age-related macular d...",What is Macular Degeneration?: Macular degener...,Macular Degeneration Age-related macular dege...,https://www.nlm.nih.gov/medlineplus/maculardeg...,3-Incomplete
2,how do you get neuromuscular disorders,115_Answer3,Myasthenia gravis is a neuromuscular disorder ...,"In people with myasthenia gravis, the body pro...",Myasthenia gravis is a neuromuscular disorder ...,"In people with myasthenia gravis, the body pro...",Myasthenia gravis: Myasthenia gravis is a neur...,Myasthenia gravis Neuromuscular disorder - my...,https://medlineplus.gov/ency/article/000712.htm,3-Incomplete
3,how do you get neuromuscular disorders,115_Answer8,Myasthenia gravis is a neuromuscular disorder ...,"In people with myasthenia gravis, the body pro...",Myasthenia gravis is a neuromuscular disorder ...,"In people with myasthenia gravis, the body pro...",Myasthenia gravis (Causes): Myasthenia gravis ...,Myasthenia gravis Neuromuscular disorder - my...,https://medlineplus.gov/ency/article/000712.htm,3-Incomplete
4,how do i stop using rasagiline,106_Answer1,"If you want to stop taking rasagiline, first t...",Do not stop taking rasagiline without talking ...,"If you want to stop taking rasagiline, first t...",Do not stop taking rasagiline without talking ...,Rasagiline: Rasagiline is used alone or in com...,Rasagiline Why is this medication prescribed?...,https://medlineplus.gov/druginfo/meds/a606017....,4-Excellent
5,how do i stop using rasagiline,106_Answer2,"If you want to stop taking rasagiline, first t...",Do not stop taking rasagiline without talking ...,Talk to your doctor about stopping your use of...,Do not stop taking rasagiline without talking ...,How should Rasagiline be used and what is the ...,Rasagiline Why is this medication prescribed?...,https://www.nlm.nih.gov/medlineplus/druginfo/m...,4-Excellent
6,how much ibuprofen is too much,132_Answer1,Prescription ibuprofen that comes as a tablet ...,Prescription ibuprofen comes as a tablet to ta...,Prescription ibuprofen that comes as a tablet ...,Prescription ibuprofen comes as a tablet to ta...,Ibuprofen: People who take nonsteroidal anti-i...,Ibuprofen IMPORTANT WARNING: People who take ...,https://medlineplus.gov/druginfo/meds/a682159....,4-Excellent
7,how much ibuprofen is too much,132_Answer2,Prescription ibuprofen that comes as a tablet ...,Prescription ibuprofen comes as a tablet to ta...,Prescription ibuprofen that comes as a tablet ...,Prescription ibuprofen comes as a tablet to ta...,How should Ibuprofen be used and what is the d...,Ibuprofen IMPORTANT WARNING: People who take ...,https://www.nlm.nih.gov/medlineplus/druginfo/m...,4-Excellent
8,how much ibuprofen is too much,132_Answer7,Prescription ibuprofen that comes as a tablet ...,Prescription ibuprofen comes as a tablet to ta...,Ibuprofen can help children with colds or mino...,Ibuprofen dosing for children: Taking ibuprofe...,Ibuprofen dosing for children: Taking ibuprofe...,Ibuprofen dosing for children Motrin Advil Su...,https://medlineplus.gov/ency/patientinstructio...,3-Incomplete
9,adrenoleukodystrophy A child I have been asked...,5_Answer2,Adrenoleukodystroph

#Making our train data compatible for BERTSUM

##ran into a problem here, thinking still </br>
now i dont want redundant queries, and summarries in case of multi-doc ext -- Model will overfit, rather i create </br>

1) Multi - doc extractive/abstractive summary csv (Query, Multi-doc ext summ, Multi-doc abstr summ) </br>
-- but dont we need the context? , okay we need the section part for extractive summary, but wont this introduce redundancy again as we will have multiple sections for the same query. -- sol: let's just combine these sections into one column, we can do this on the basis of query? yes we did it </br> for abstractive summary we need to have the whole article
2) Single - doc extr summ csv ( query, single doc extr summ, section) and we dont need section for this just article also we will have the rating of the summary as well

In [7]:
import pandas as pd

# Load dataset
file_path = "/content/cleaned_train_data_with_ids.csv"
df = pd.read_csv(file_path)

# Group by Query and Aggregate Sections
multi_doc_df_ext = df.groupby(["Query", "Multi-Doc Extractive Summary"], as_index=False).agg({
    "Detailed Section": lambda x: " ".join(x)  # Combine all sections per query
})

# Save to CSV
multi_doc_df_ext.to_csv("/content/multi_doc_summaries_with_aggregated_sections.csv", index=False)

print("Multi-Document Extractive Summary CSV Created with Combined Sections!")

multi_doc_df_ext[multi_doc_df_ext["Query"] == "how much oxazepam could cause an overdose?"]


Multi-Document Extractive Summary CSV Created with Combined Sections!


,Query,Multi-Doc Extractive Summary,Detailed Section
59,how much oxazepam could cause an overdose?,Oxazepam is a medicine used to treat anxiety a...,What is Oxazepam overdose?: Oxazepam is a medi...


In [21]:
import pandas as pd

# Load dataset
file_path = "/content/cleaned_validation_data_with_ids.csv"
df = pd.read_csv(file_path)

# Group by Query and Aggregate Sections
multi_doc_df_ext_val = df.groupby(["Query", "Multi-Doc Extractive Summary"], as_index=False).agg({
    "Detailed Section": lambda x: " ".join(x)  # Combine all sections per query
})

# Save to CSV
multi_doc_df_ext_val.to_csv("/content/validation_multi_doc_summaries_with_aggregated_sections.csv", index=False)

print("Multi-Document Extractive Summary CSV Created with Combined Sections for Validation Dataset!")

multi_doc_df_ext[multi_doc_df_ext["Query"] == "subjective vertigo Can macular degeneration in only one eye cause dizziness?"]

Multi-Document Extractive Summary CSV Created with Combined Sections for Validation Dataset!


,Query,Multi-Doc Extractive Summary,Detailed Section
13,subjective vertigo Can macular degeneration in...,"Dizziness has many possible causes, including ...",Dizziness: Dizziness is a term used to describ...


In [26]:
file_path = "/content/validation_multi_doc_summaries_with_aggregated_sections.csv"
df1 = pd.read_csv(file_path)
print(df1.shape)

(16, 3)


In [25]:
file_path = "/content/multi_doc_summaries_with_aggregated_sections.csv"
df1 = pd.read_csv(file_path)
print(df1.shape)

(110, 3)


In [36]:
# heart of the program, here we see if the ext summ is relevent to the query, then we also see if the data is taken from the combined section
import pandas as pd
import nltk
import pickle
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util  # Sentence Embeddings

nltk.download("punkt")

# 📌 Load sentence embedding model for semantic similarity
model = SentenceTransformer("all-MiniLM-L6-v2")  # Fast & efficient for similarity comparison

# 📌 Paths to datasets
train_file_path = "/content/multi_doc_summaries_with_aggregated_sections.csv"
val_file_path = "/content/validation_multi_doc_summaries_with_aggregated_sections.csv"

# ✅ Function to check if a sentence is semantically similar to the extractive summary OR relevant to the query
def is_relevant(sent, summary_sentences, query, threshold=0.8):
    """
    Checks if a sentence is semantically similar to any extractive summary sentence OR relevant to the query.
    Uses cosine similarity between sentence embeddings.
    """
    sent_embedding = model.encode(sent, convert_to_tensor=True)
    summary_embeddings = model.encode(summary_sentences, convert_to_tensor=True)
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute similarity with extractive summary
    summary_similarities = util.pytorch_cos_sim(sent_embedding, summary_embeddings)

    # Compute similarity with query
    query_similarity = util.pytorch_cos_sim(sent_embedding, query_embedding).item()

    # Mark sentence as important if it's similar to the summary OR relevant to the query
    return bool((summary_similarities > threshold).any() or query_similarity > threshold)

# ✅ Function to preprocess dataset
def preprocess_data(file_path):
    df = pd.read_csv(file_path)

    text_inputs = []
    labels = []

    for index, row in df.iterrows():
        query = row["Query"]  # The query is now included in processing
        summary = row["Multi-Doc Extractive Summary"]
        sections = row["Detailed Section"]

        # Tokenizing sentences
        full_text = query + " " + sections  # Combine query + sections
        sentences = sent_tokenize(full_text)
        summary_sentences = sent_tokenize(summary)

        # Labels: Use semantic similarity for both summary and query relevance
        label = [1 if is_relevant(sent, summary_sentences, query) else 0 for sent in sentences]

        text_inputs.append(sentences)
        labels.append(label)

    return text_inputs, labels

# 📌 Preprocess both training and validation datasets
train_texts, train_labels = preprocess_data(train_file_path)
val_texts, val_labels = preprocess_data(val_file_path)

# ✅ Save processed data
processed_data = {
    "train_texts": train_texts,
    "train_labels": train_labels,
    "val_texts": val_texts,
    "val_labels": val_labels
}

# Debugging output to check labels
print("Sample Training Labels:", train_labels[:5])
print("Sample Validation Labels:", val_labels[:5])

# Save processed data to file
with open("/content/processed_bertsum_data.pkl", "wb") as f:
    pickle.dump(processed_data, f)

print("✅ Preprocessing Complete! Train & Validation Data saved for training.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sample Training Labels: [[1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1], [1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [37]:
import pickle

# Load preprocessed data
with open("/content/processed_bertsum_data.pkl", "rb") as f:
    processed_data = pickle.load(f)

# Extract train & validation data
train_texts, train_labels = processed_data["train_texts"], processed_data["train_labels"]
val_texts, val_labels = processed_data["val_texts"], processed_data["val_labels"]

print(f" Data Loaded! Training samples: {len(train_texts)}, Validation samples: {len(val_texts)}")


 Data Loaded! Training samples: 110, Validation samples: 16


In [38]:
import torch
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Convert text to tokenized input for BERTSUM
def encode_sentences(sentences, tokenizer, max_length=512):
    input_ids = []
    attention_masks = []

    for sent in sentences:
        encoded = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids.append(encoded["input_ids"])
        attention_masks.append(encoded["attention_mask"])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Encode training and validation data
train_input_ids, train_attention_masks = [], []
val_input_ids, val_attention_masks = [], []

for text in train_texts:
    input_id, attn_mask = encode_sentences(text, tokenizer)
    train_input_ids.append(input_id)
    train_attention_masks.append(attn_mask)

for text in val_texts:
    input_id, attn_mask = encode_sentences(text, tokenizer)
    val_input_ids.append(input_id)
    val_attention_masks.append(attn_mask)

# Convert labels to tensors
train_label_ids = [torch.tensor(label, dtype=torch.float) for label in train_labels]
val_label_ids = [torch.tensor(label, dtype=torch.float) for label in val_labels]

# Convert lists to tensors
train_inputs = torch.cat(train_input_ids, dim=0)
train_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.cat(train_label_ids, dim=0)

val_inputs = torch.cat(val_input_ids, dim=0)
val_masks = torch.cat(val_attention_masks, dim=0)
val_labels = torch.cat(val_label_ids, dim=0)

print(f"✅ Tokenization Complete! Train shape: {train_inputs.shape}, Val shape: {val_inputs.shape}")


✅ Tokenization Complete! Train shape: torch.Size([12519, 512]), Val shape: torch.Size([1574, 512])


In [39]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Define batch size
batch_size = 32

# Create training dataset & dataloader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create validation dataset & dataloader
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

print(f" DataLoaders Ready! Train batches: {len(train_dataloader)}, Val batches: {len(val_dataloader)}")


 DataLoaders Ready! Train batches: 392, Val batches: 50


In [40]:
from transformers import BertForSequenceClassification

# Load BERTSUM model (Pretrained)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,  # Binary classification (sentence in summary: 1, otherwise: 0)
    output_attentions=False,
    output_hidden_states=False
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f" Model Loaded! Running on: {device}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Model Loaded! Running on: cuda


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from torch.optim import lr_scheduler
from tqdm import tqdm
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Define loss function
loss_fn = nn.BCEWithLogitsLoss()

# Learning rate scheduler: Reduce LR when validation loss stops improving
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)

# Early stopping settings
early_stopping_patience = 2
best_val_loss = float("inf")
epochs_no_improve = 0
checkpoint_path = "best_model_checkpoint.pth"

# 🚀 Training function with Checkpoints & Overfitting Prevention
def train_model(model, train_dataloader, val_dataloader, epochs=10):
    global best_val_loss, epochs_no_improve

    for epoch in range(epochs):
        print(f"\n🚀 Epoch {epoch+1}/{epochs} Starting...")

        # Training Phase
        model.train()
        total_train_loss = 0
        train_progress = tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}")

        for batch in train_progress:
            b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

            # Convert labels to one-hot encoding
            b_labels = F.one_hot(b_labels.to(torch.int64), num_classes=2).float().to(device)

            model.zero_grad()

            # Forward pass
            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits

            # Compute loss
            loss = loss_fn(logits, b_labels)

            # Backward pass
            loss.backward()

            # Gradient Clipping (prevents exploding gradients)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_train_loss += loss.item()
            train_progress.set_postfix({"Loss": loss.item()})

        # Validation Phase
        model.eval()
        total_val_loss = 0
        val_progress = tqdm(val_dataloader, desc=f"Validation Epoch {epoch+1}")

        with torch.no_grad():
            for batch in val_progress:
                b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

                # Convert labels to one-hot encoding
                b_labels = F.one_hot(b_labels.to(torch.int64), num_classes=2).float().to(device)

                outputs = model(b_input_ids, attention_mask=b_input_mask)
                logits = outputs.logits

                val_loss = loss_fn(logits, b_labels)
                total_val_loss += val_loss.item()
                val_progress.set_postfix({"Loss": val_loss.item()})

        avg_train_loss = total_train_loss / len(train_dataloader)
        avg_val_loss = total_val_loss / len(val_dataloader)

        print(f"\n✅ Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        # Step the learning rate scheduler
        scheduler.step(avg_val_loss)

        # Save best model checkpoint
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0  # Reset counter
            torch.save(model.state_dict(), checkpoint_path)
            print(f"📌 New Best Model Saved at {checkpoint_path} (Val Loss: {avg_val_loss:.4f})")
        else:
            epochs_no_improve += 1
            print(f"⚠️ No Improvement for {epochs_no_improve} Epochs")

        # Early stopping
        if epochs_no_improve >= early_stopping_patience:
            print(f"⛔ Stopping Early! Best Validation Loss: {best_val_loss:.4f}")
            break

    print("\n🎉 Training Complete! 🚀")

# Run training with batch size 16
train_model(model, train_dataloader, val_dataloader, epochs=10)

# Save final model
torch.save(model.state_dict(), "bert_extractive_summarizer.pth")
print("✅ Final Model Saved as bert_extractive_summarizer.pth")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



🚀 Epoch 1/10 Starting...


Training Epoch 1:  68%|██████▊   | 268/392 [12:43<05:53,  2.85s/it, Loss=0.334]

In [16]:
import os

if os.path.exists("bert_extractive_summarizer.pth"):
    model.load_state_dict(torch.load("bert_extractive_summarizer.pth", map_location=device))
    print("✅ Model Loaded Successfully!")
else:
    print("⚠️ Model file not found! Check if it was saved correctly.")

⚠️ Model file not found! Check if it was saved correctly.


In [15]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BERT model architecture
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Load saved weights
model.load_state_dict(torch.load("bert_extractive_summarizer.pth", map_location=device))

# Move model to GPU & set to eval mode
model.to(device)
model.eval()

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

print("✅ Model and Tokenizer Reloaded Successfully!")

# 🔎 Function to Make Predictions
def predict_summary(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    confidence = torch.softmax(logits, dim=1)[0]
    predicted_class = torch.argmax(logits, dim=1).item()

    return f"Prediction: {'Important Sentence' if predicted_class == 1 else 'Not Important'} (Confidence: {confidence[predicted_class]:.2f})"

# Example Test
test_text = "Oxazepam is used to treat anxiety and symptoms of alcohol withdrawal."
print(f"🚀 {predict_summary(test_text)}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-15-3b7581135674>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We rec

FileNotFoundError: [Errno 2] No such file or directory: 'bert_extractive_summarizer.pth'

#Saving the model